# Let's explore the RFX MDSplus tree

## Importing and setting up stuff

In [1]:
import MDSplus as mds
import numpy as np
import matplotlib.pyplot as plt
import sys, os, time, random
from tqdm import tqdm
import h5py as h5
print(f'Python version: {sys.version}')
print(f'MDSplus version: {mds.__version__}')
np.set_printoptions(precision=3, suppress=True)

Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
MDSplus version: 1.0.0


In [2]:
#color the terminal output
def pick_random_color():
    return '\033[38;5;{}m'.format(random.randint(8, 230))
ENDC = '\033[0m'
ERR = '\033[91m'+ 'ERR: '
OK = '\033[92m'
WARN = '\033[93m'+ 'WARN: '

In [3]:
# define the shot number and tree
SHOT = 30810
rfx = mds.Tree('rfx', SHOT, 'readonly') # open the tree read-only

## Traversing the tree

In [4]:
# traverse the tree, use MAX_DEPTH to limit the depth of the tree to traverse
# othwerwise the script will run for about 10 minutes
MAX_DEPTH = 5 # maximum depth of the tree to traverse
COLORS = [pick_random_color() for i in range(MAX_DEPTH)]

usage_depth, usage_breadth = {},{}

def traverse_tree_depth_first(max_depth, node, level=0, path='', node_type='child'):
    try: 
        if level >= max_depth: return # stop if the maximum depth is reached
        if node_type == 'child': node_name = node.node_name.upper()
        elif node_type == 'member': node_name = node.node_name.lower()
        else: raise
        path = path + '/' + COLORS[level] + node_name + ENDC # add the node name
        print(f'{path}:{node.decompile()}') 
        # get the usage/type of the node
        try: usage_depth[str(node.usage)] += 1
        except: usage_depth[str(node.usage)] = 1
        # go through the children and members of the node
        for child in node.getChildren(): # get the children of the node
            traverse_tree_depth_first(max_depth, child, level + 1, path, 'child')
        for member in node.getMembers(): # get the members of the node
            traverse_tree_depth_first(max_depth, member, level + 1, path, 'member')
    except Exception as e:
        print(path + 'ERR:' + str(e))
        pass

# do the same but without recursion
def traverse_tree_breadth_first(max_depth, head_node):
    curr_nodes = [head_node]
    for d in range(max_depth):
        print('Depth:', d)
        next_nodes = []
        for node in curr_nodes:
            try:
                preprint = COLORS[d] + "   " * d + node.node_name + ENDC
                print(f'{preprint}:{node.decompile()}') # print the node
                # get the usage/type of the node
                try: usage_breadth[str(node.usage)] += 1
                except: usage_breadth[str(node.usage)] = 1
                # get the children of the node
                for child in node.getChildren():
                    next_nodes.append(child)
                # get the members of the node
                for member in node.getMembers():
                    next_nodes.append(member)
            except: pass
        curr_nodes = next_nodes
        
head_node = rfx.getNode('\\TOP.RFX') # get the top node

# test the functions, uncomment to run
# traverse_tree_depth_first(MAX_DEPTH, head_node) # traverse the tree depth-first
# traverse_tree_breadth_first(MAX_DEPTH, head_node) # traverse the tree breadth-first

In [5]:
print(f'Usage depth: {usage_depth}')
print(f'Usage breadth: {usage_breadth}')

Usage depth: {}
Usage breadth: {}


previous cell full depth: 'STRUCTURE': 8776, 'SUBTREE': 78, 'DEVICE': 642, 'ACTION': 1098, 'NUMERIC': 47760, 'TEXT': 17269, 'SIGNAL': 20904, 'ANY': 29, 'AXIS': 215

In [6]:
print(f'top nodes: {[n.node_name for n in head_node.getChildren()]}')

top nodes: ["ACTIONS", "DIAG", "EDA", "MHD", "SETUP", "STC", "VERSIONS"]


## Exploring Signals

In [7]:
search_space = '\\TOP.RFX.MHD.***' # *** means all nodes at this level
# search_space = '\\TOP.RFX.EDA.***' # * means all nodes at this level
# search_space = '\\TOP.RFX.***' # whole rfx tree
signal_nodes = rfx.getNodeWild(search_space, 'Signal') # get all nodes with the name 'Signal'
print(f'Found {len(signal_nodes)} of the type Signal in the search space {search_space}')

Found 5491 of the type Signal in the search space \TOP.RFX.MHD.***


In [8]:
# filter out the nodes without the data
data_signals = []
for node in tqdm(signal_nodes, leave=False):
    try: data = node.data(); data_signals.append(node)
    except: pass
print(f'Found {len(data_signals)}/{len(signal_nodes)} signals with data')

Found 4356/5491 signals with data


In [9]:
# keep only the signals with raw data
raw_signals = []
for node in tqdm(signal_nodes, leave=False):
    try: data = node.raw_of().data(); raw_signals.append(node)
    except: pass
print(f'Found {len(raw_signals)}/{len(data_signals)} signals with raw data')


Found 3971/4356 signals with raw data


In [10]:
# extract data from the signals and plot them
MAX_LOAD = 0 #10 #np.inf
MAX_LOAD = min(MAX_LOAD, len(raw_signals))
# select MAX_LOAD random signals
signals = random.sample(raw_signals, MAX_LOAD)
for node in (signals):
    signal = node.data()
    times = node.dim_of().data()
    unit = node.getUnits()
    full_path = node.getFullPath()
    try: node_help = node.getHelp()
    except: node_help = ''
    if signal.shape != times.shape:
        print(f'{full_path} has mismatched signal and time shapes')
        continue
    # plot the signal
    plt.figure()
    plt.plot(times, signal)
    plt.title(f'{full_path} [{unit}]\n{node_help}')
    plt.xlabel('Time [s]')
    plt.ylabel('Signal')
    plt.show()

## Exploring Text

In [11]:
text_nodes = rfx.getNodeWild(search_space, 'Text') # get all the 'TEXT' nodes
print(f'Found {len(text_nodes)} of the type Text in the search space {search_space}')
# print all the text nodes
for node in text_nodes:
    try: print(f'{node.getFullPath()}={node.data()}')
    except: pass

Found 2669 of the type Text in the search space \TOP.RFX.MHD.***
\RFX::TOP.RFX.MHD.MHD_AC:VME:CONTROL:COMMENT=#1 SlinkyRot #2 Zanca2008 #3 Hybrid 2010
\RFX::TOP.RFX.MHD.MHD_AC:VME:CONTROL:FEEDFORW=DISABLED
\RFX::TOP.RFX.MHD.MHD_AC:VME:CONTROL:ROUTINE_NAME=MhdAc
\RFX::TOP.RFX.MHD.MHD_AC:VME:CONTROL:VERSION=2.3.0
\RFX::TOP.RFX.MHD.MHD_AC:VME:CONTROL:VME_IP=150.178.34.29
\RFX::TOP.RFX.MHD.MHD_AC:VME:CURR_FF:DESCRIPTION=Null waveforms
\RFX::TOP.RFX.MHD.MHD_AC:VME:CURR_FF:VME_IP=150.178.34.29
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:ADC:TR10_1:CLOCK_MODE=EXTERNAL
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:ADC:TR10_1:COMMENT=
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:ADC:TR10_1:IP_ADDR=
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:ADC:TR10_1:SW_MODE=LOCAL
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:ADC:TR10_1:TRIG_EDGE=RISING
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:ADC:TR10_1:TRIG_MODE=EXTERNAL
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:ADC:TR10_1:USE_TIME=TRUE
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:ADC:TR10_10:CLOCK_MODE=EXTERNAL
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:ADC:TR10

## Convert tree in HDF5 format

In [12]:
HDF5_FILE = f'rfx_{SHOT}.hdf5'

In [13]:
def convert_tree_hdf5(head_node, new_tree_file):
    #explore all the tree and save the data to the hdf5 file
    max_depth = 4
    with h5.File(new_tree_file, 'w') as f:
        curr_nodes = [(head_node,'child')]
        tot_nodes_explored, valid_nodes, signal_nodes = 0, 0, 0
        for d in range(max_depth):
            print('Depth:', d)
            next_nodes = []
            for node, node_type in curr_nodes:
                tot_nodes_explored += 1
                full_path = str(node.getFullPath())[10:]#.lower() # remove the \\TOP.RFX prefix
                full_path = full_path.replace('.', '/') #convert . to /
                
                try: # get the children and members of the node
                    for child in node.getChildren(): next_nodes.append((child, 'child'))
                    for member in node.getMembers(): next_nodes.append((member, 'member'))
                except: pass # do nothing if the node has no children or members
                
                usage = str(node.usage)
                
                # check if the node has data
                try: data = node.data()
                except: data = None
                
                #check if the node has time
                try: times = node.dim_of().data()
                except: times = None
                
                # VARIOUS CHECKS
                if usage == 'SIGNAL' and data is None: # signal without data
                    print(f'{full_path} is signal but has no data'); continue
                if usage == 'SIGNAL' and data is not None and times is None: # signal with data but no time
                    print(f'{full_path} is signal with data but no time: data: {data}'); continue
                if usage == 'NUMERIC' and data is None: # numeric without data
                    print(f'{full_path} is numeric but has no data'); continue
                if data is not None and times is not None: # node with data and time
                    if data.shape != times.shape: # mismatched signal and time shapes, multidimensional signals are notable exceptions, but for now we skip them
                        print(f'{full_path} has mismatched signal and time shapes, {data.shape} != {times.shape}'); continue
                    if usage != 'SIGNAL': # not a signal but has data and time
                        print(f'{full_path} is not signal but has data and time'); continue
                        
                valid_nodes += 1 # valid node
                
                if usage in ['SIGNAL', 'NUMERIC']: # save the data to the hdf5 file
                    signal_nodes += 1
                    if data is not None: f.create_dataset(full_path, data=data)
                    if times is not None: f.create_dataset(full_path + '/times', data=times)
                
                # print(f'{full_path}:{usage.lower()}')
                
                # print(f'{valid_nodes}/{tot_nodes_explored} with usage {usage} and data {data}')
                
            curr_nodes = next_nodes # update the current nodes
    
    print(f'Valid nodes: {valid_nodes}/{tot_nodes_explored}, {valid_nodes/tot_nodes_explored*100:.2f}%')
    print(f'Signal nodes: {signal_nodes}/{valid_nodes}, {signal_nodes/valid_nodes*100:.2f}%')
# test the function
convert_tree_hdf5(rfx.getNode('\\RFX::TOP.RFX'), HDF5_FILE)

Depth: 0
Depth: 1
Depth: 2
RFX/VERSIONS:URUNS is numeric but has no data
Depth: 3
RFX/SETUP:UNITS:A_PARS is not signal but has data and time
RFX/SETUP:UNITS:B_PARS is not signal but has data and time
RFX/STC:DIO2_1:REC_EVENTS is not signal but has data and time
RFX/STC:DIO2_1:REC_TIMES is not signal but has data and time
RFX/STC:DIO2_2:REC_EVENTS is not signal but has data and time
RFX/STC:DIO2_2:REC_TIMES is not signal but has data and time
RFX/STC:DIO2_3:REC_EVENTS is not signal but has data and time
RFX/STC:DIO2_3:REC_TIMES is not signal but has data and time
RFX/STC:DIO2_4:REC_EVENTS is not signal but has data and time
RFX/STC:DIO2_4:REC_TIMES is not signal but has data and time
RFX/STC:DIO2_5:REC_EVENTS is not signal but has data and time
RFX/STC:DIO2_5:REC_TIMES is not signal but has data and time
RFX/STC:DIO2_6:REC_EVENTS is not signal but has data and time
RFX/STC:DIO2_6:REC_TIMES is not signal but has data and time
RFX/STC:DIO2_7:REC_EVENTS is not signal but has data and time


In [20]:
# version 2, let's use recursion
MAX_DEPTH = 6


with h5.File(HDF5_FILE, 'w') as hdf:

    def hasData(node):
        try: _ = node.data(); return True
        except: return False

    def hasTime(node):
        try: _ = node.dim_of().data(); return True
        except: return False
        
    def hasChildren(node):
        try: return len(node.getChildren()) > 0
        except: return False

    def hasMembers(node):
        try: return len(node.getMembers()) > 0
        except: return False

    def explore_tree(max_depth, node, level=0, path=''):
        if level >= max_depth: return # stop if the maximum depth is reached
        is_child = node.isChild() # check if the node is a child
        is_member = node.isMember() # check if the node is a member
        assert (is_child or is_member) and not (is_child and is_member) # check if the node is either a child or a member
        has_children = hasChildren(node) # check if the node has children
        has_members = hasMembers(node) # check if the node has members
        has_time = hasTime(node) # check if the node has time
        has_data = hasData(node) # check if the node has data
        nname = node.node_name.upper() if is_child else node.node_name.lower() # get the node name
        npath = node.getFullPath() # get the full path of the node
        nusage = str(node.usage) # get the usage/type of the node
        path = path + '/' + nname # add the node name to the path
        # print(path)
        
        # associate an hdf5 file
        if has_children or has_members: # create a group for the node
            group = hdf.create_group(path)
            group.attrs['usage'] = nusage
        
        
        if has_data and has_children: print(f'{ERR}NODE {path} has DATA and CHILDREN: data: {node.data()}, CHILDREN: {node.getChildren()}{ENDC}')
        if has_data and has_members: print(f'{WARN}node {path} has data and members: data: {node.data()}, members: {node.getMembers()}{ENDC}')

        # if has_data: # save the data to the hdf5 file
        #     data = node.data()
        #     hdf.create_dataset(path, data=data)
            
        
        
        
        
        if has_children: # recursively go through the children of the node
            for child in node.getChildren(): explore_tree(max_depth, child, level + 1, path)
        if has_members: # recursively go through the members of the node
            for member in node.getMembers(): explore_tree(max_depth, member, level + 1, path)

    explore_tree(MAX_DEPTH, rfx.getNode('\\TOP.RFX')) # traverse the tree depth-first



node /RFX/DIAG/DMOSS/RESULTS/CALIBRATION/i0 has data and members: data: [0.33  0.213 0.037 0.472], members: [352321547]
node /RFX/DIAG/DMOSS/RESULTS/CALIBRATION/phi0 has data and members: data: [105.066  97.018  98.092  84.7  ], members: [352321551]
node /RFX/DIAG/DMOSS/RESULTS/CALIBRATION/phi1 has data and members: data: [151.406 151.795 150.917 145.758], members: [352321554,352321552]
node /RFX/DIAG/DMOSS/RESULTS/CALIBRATION/zeta has data and members: data: [0.583 0.499 0.613 0.639], members: [352321546]
node /RFX/DIAG/DSFM/PARAMETERS/PARAM_DSFM/hv_1 has data and members: data: 0.004809999838471413, members: [587202703]
node /RFX/DIAG/DSFM/SIGNALS/he_7 has data and members: data: [ 0. -0. -0. ...  0.  0.  0.], members: [587202701]
node /RFX/DIAG/DSFM/SIGNALS/o2_9 has data and members: data: [0.021 0.021 0.021 ... 0.021 0.021 0.021], members: [587202700,587202669,587202679]


In [21]:
# print the tree structure
def h5_tree(vals, pre='', mid_syms=('├────','│     '), end_syms=('└────','      ')):
    for i, (key, val) in enumerate(vals.items()):
        s1, s2 = end_syms if i == len(vals)-1 else mid_syms
        if type(val) == h5.Group: 
            print(f'{pre}{s1} {key}') 
            h5_tree(val, f'{pre}{s2}', mid_syms, end_syms)
        else: print(f'{pre}{s1} {key} [{val.shape} {val.dtype}]')

In [22]:
with h5.File(HDF5_FILE, 'r') as f: h5_tree(f)

└──── RFX
      ├──── ACTIONS
      ├──── DIAG
      │     ├──── A
      │     │     ├──── PARAMS
      │     │     ├──── PROGRAMS
      │     │     │     ├──── BT_SIGNALS
      │     │     │     ├──── EMRA
      │     │     │     └──── EQ_TOR
      │     │     ├──── RESULTS
      │     │     │     ├──── BEPEQRA
      │     │     │     ├──── BEPERA
      │     │     │     ├──── BEPERP
      │     │     │     ├──── DFBEPERA
      │     │     │     ├──── DFLU_SIGNALS
      │     │     │     │     ├──── POL
      │     │     │     │     └──── TOR
      │     │     │     ├──── DFTAUERA
      │     │     │     ├──── ELIBRA
      │     │     │     ├──── ELIBRP
      │     │     │     ├──── EMRA
      │     │     │     ├──── EMRP
      │     │     │     ├──── ENEL
      │     │     │     ├──── EQFLU
      │     │     │     ├──── EQFLU_BR
      │     │     │     ├──── EQ_TOR
      │     │     │     ├──── ETASP
      │     │     │     ├──── INCO2MRA
      │     │     │     ├──── INCO2SRA
      